# Inference

This notebook is for development/testing of the inference server (`inference.py`).

It is similar to the `training/training.ipynb` notebook,
but uses a live page instead of one from the dataset.

In [ ]:
import awe.utils
awe.utils.init_notebook()

In [ ]:
import base64
import json
import os

In [ ]:
import awe.data.set.live
import awe.data.visual.exploration
import awe.inference
import awe.training.versioning
import awe.training.params
import awe.training.trainer
awe.utils.reload('awe', exclude=['awe.data.glove'])

In [ ]:
# Load trained version from a checkpoint.
version = awe.training.versioning.Version.get_latest()
checkpoint = version.get_checkpoints()[-1]
params = awe.training.params.Params.load_version(version)
params.patch_for_inference()
print(f'{params=}')
trainer = awe.training.trainer.Trainer(params)
trainer.init_features()
trainer.restore_checkpoint(checkpoint)
trainer.restore_features()
trainer.create_model()
trainer.restore_model()

In [ ]:
# Load inputs if they exist (can be saved from the visual extractor, see
# `docs/dev/tips.md`).
SCREENSHOT_PATH = 'data/screenshot.txt'
if os.path.exists(SCREENSHOT_PATH):
    with open(SCREENSHOT_PATH, mode='r', encoding='utf-8') as f:
        screenshot = base64.b64decode(f.read())
    print('Loaded screenshot.')
else:
    screenshot = None

HTML_PATH = 'data/page.html'
if os.path.exists(HTML_PATH):
    with open(HTML_PATH, mode='r', encoding='utf-8') as f:
        html_text = f.read()
    print('Loaded HTML.')
else:
    html_text = '<html><body><b>b</b><i>i</i></body></html>'

VISUALS_PATH = 'data/visuals.json'
if os.path.exists(VISUALS_PATH):
    with open(VISUALS_PATH, mode='r', encoding='utf-8') as f:
        visuals_data = json.load(f)
    print('Loaded visuals.')
else:
    visuals_data = {"/html":{"/body":{"/b":{"/text()":{"box":[0, 0, 1, 1]}},"/i":{"/text()":{"box":[1, 1, 2, 2]}}}}}

In [ ]:
# Mock of a live page (working example needs visuals extracted by the Node.js
# extractor).
page = awe.data.set.live.Page(
    index=0,
    url='https://example.com/',
    html_text=html_text,
    visuals_data=visuals_data,
    screenshot=screenshot,
)

In [ ]:
# Prepare features for the page.
run = trainer.create_run([page], desc='live')

In [ ]:
# Predict results for the page.
preds = trainer.predict(run)
trainer.decode(preds)

In [ ]:
# Postprocess.
preds = awe.inference.postprocess(preds)
trainer.decode(preds)

In [ ]:
# Plot screenshot with predicted nodes highlighted.
if screenshot is not None:
    page.fill_labels(trainer, preds)
    awe.inference.plot_screenshot(page)